# Apache Spark com Delta Lake

Este notebook demonstra o uso de Apache Spark com Delta Lake para manipulação de dados estruturados em um cenário de locadora de veículos.

## Cenário do Projeto

O sistema gerencia dois tipos de entidades:
- **Clientes**: contendo informações pessoais como nome, e-mail e telefone.
- **Carros**: contendo a placa e o cliente responsável.

### Modelo ER Simplificado
Cliente (1) --- (N) Carros

![modelo ER](https://i.imgur.com/MmGpRkh.png)

### Fonte dos Dados
Os dados utilizados neste projeto são fictícios, inspirados em sistemas de locadoras de veículos

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType

from delta import *

In [ ]:
# Create SparkSession
spark = (
    SparkSession
    .builder
    .master("local[*]")
    .config("spark.jars.packages", "io.delta:delta-spark_2.12:3.2.0")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .getOrCreate()
)

In [ ]:
spark

## Criação das Tabelas Delta

In [ ]:
spark.sql("""
  CREATE TABLE cliente_delta (
    id INT,
    nome STRING,
    email STRING,
    telefone STRING
  ) USING delta
""")

spark.sql("""
  CREATE TABLE carro_delta (
    id INT,
    placa STRING,
    cliente_id INT
  ) USING delta
""")

In [ ]:
spark.sql("SELECT * FROM cliente_delta").show()

In [ ]:
spark.sql("SELECT * FROM carro_delta").show()

In [ ]:
from delta.tables import DeltaTable

cliente = DeltaTable.forPath(spark, "./spark-warehouse/cliente_delta")

carro = DeltaTable.forPath(spark, "./spark-warehouse/carro_delta")

In [ ]:
cliente.history().show()

In [ ]:
carro.history().show()

## Inserção de Dados nas Tabelas Delta

In [ ]:
spark.sql("""
  INSERT INTO cliente_delta VALUES
  (1, 'Ana Souza', 'ana@email.com', '(11) 91234-5678'),
  (2, 'Bruno Lima', 'bruno@email.com', '(21) 99876-5432'),
  (3, 'Carla Mendes', 'carla@email.com', '(31) 98765-4321')
""")

spark.sql("""
  INSERT INTO carro_delta VALUES
  (1, 'XYZ1J34', 1),
  (2, 'RLC5B93', 2),
  (3, 'ABC9Z88', 1)
""")

In [ ]:
spark.sql("SELECT * FROM cliente_delta").show()
spark.sql("SELECT * FROM carro_delta").show()

## Comandos de Manipulação: UPDATE e DELETE

In [ ]:
spark.sql("""
  UPDATE cliente_delta
  SET telefone = '(11) 90000-0000'
  WHERE id = 1
""")

spark.sql("""
  DELETE FROM carro_delta
  WHERE id = 2
""")

## Visualização dos Dados e Histórico Delta Lake

In [ ]:
spark.sql("SELECT * FROM cliente_delta").show()
spark.sql("SELECT * FROM carro_delta").show()


cliente.history().show(truncate=False)
carro.history().show(truncate=False)